In [1]:
def Lorentz(E, An, Br, En, einf=1.0):
    """
    Compute the Lorentz oscillator dielectric function.

    Parameters:
    - E:     array of photon energies (eV)
    - An:    oscillator amplitude
    - Br:    broadening parameter
    - En:    resonance energy
    - einf:  high-frequency permittivity (default 1.0)

    Returns:
    - DataFrame with columns:
        • 'Energy (eV)'  
        • 'e1' (real dielectric)  
        • 'e2' (imaginary dielectric)  
        • 'e'  (complex dielectric)
    """

    # 1) Ensure energies are in an array
    E = np.array(E)

    # 2) Compute complex dielectric: ε = ε_inf + (An·Br·En) / (En² − E² − i·Br·E)
    numerator   = An * Br * En
    denominator = En**2 - E**2 - 1j * Br * E
    dielectric  = einf + numerator / denominator

    # 3) Split into real (e1) and imaginary (e2) parts
    e1 = dielectric.real
    e2 = dielectric.imag

    # 4) Build DataFrame
    df = pd.DataFrame({
        'Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 5) Annotate name and source_info for metadata
    df.name = f"A_{An}_Br_{Br}_En_{En}_Einf_{einf}"
    df.attrs["source_info"] = {
        "model":   "Lorentz",
        "An":      An,
        "Br":      Br,
        "En":      En,
        "Einf":    einf
    }

    return df

In [2]:
def drude_epsilon(E, rho_n, tau_fs):
    """
    Compute Drude-model dielectric function for free carriers.

    Parameters:
    - E:       array of photon energies (eV)
    - rho_n:   material resistivity (Ohm·cm)
    - tau_fs:  carrier scattering time (fs)

    Returns:
    - DataFrame with:
        • 'Energy (eV)'  
        • 'e1' (real part of ε)  
        • 'e2' (imaginary part of ε)  
        • 'e'  (complex dielectric function)
    """
    # 1) Physical constants
    hbar     = 6.582119569e-16   # Reduced Planck constant [eV·s]
    eps0     = 8.854e-14         # Vacuum permittivity [F/cm]
    tau      = tau_fs * 1e-15    # Convert scattering time from fs to s

    # 2) Drude numerator & denominator
    #    ε(ω) = -ħ² / [ε₀·ρ_n·(τ·E² + i·ħ·E)]
    numerator   = -hbar**2
    denominator = eps0 * rho_n * (tau * E**2 + 1j * hbar * E)

    # 3) Compute complex dielectric function
    dielectric = numerator / denominator

    # 4) Separate real and imaginary parts
    e1 = dielectric.real
    e2 = dielectric.imag

    # 5) Assemble results into a DataFrame
    df = pd.DataFrame({
        'Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 6) Annotate metadata for traceability
    df.name = f"Drude_rho{rho_n}_tau{tau_fs}"
    df.attrs["source_info"] = {
        "model":   "Drude",
        "rho_n":   rho_n,
        "tau_fs":  tau_fs
    }

    return df

In [3]:
def Sellmeier(A_uv, A_ir, En, e_inf, E):
    """
    Compute the real dielectric function using the Sellmeier model.

    Parameters:
    - A_uv:   UV absorption strength coefficient
    - A_ir:   IR absorption strength coefficient
    - En:     resonance energy (eV)
    - e_inf:  high-frequency permittivity
    - E:      array of photon energies (eV)

    Returns:
    - DataFrame with:
        • 'Energy (eV)'  
        • 'e1' (real dielectric)  
        • 'e2' (imaginary part, zero)  
        • 'e'  (complex dielectric = e1 + 0j)
    """
    # 1) Compute components
    UV_dielectric = A_uv / (En**2 - E**2)
    IR_dielectric = -A_ir / (E**2)
    e1 = UV_dielectric + IR_dielectric + e_inf

    # 2) Imaginary part is zero for Sellmeier
    e2 = np.zeros_like(E)
    dielectric = e1 + 0j

    # 3) Build DataFrame
    df = pd.DataFrame({
        'Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 4) Add metadata
    df.attrs["source_info"] = {
        "model":  "Sellmeier",
        "A_uv":   A_uv,
        "A_ir":   A_ir,
        "En":     En,
        "e_inf":  e_inf
    }
    df.name = f"Sellmeier_Auv_{A_uv}_Air_{A_ir}_En_{En}_Einf_{e_inf}"

    return df

In [4]:
def sumosscilator(df_list):
    """
    Combine multiple oscillator DataFrames into a single composite dielectric.

    Parameters:
    - df_list: list of DataFrames, each with:
        • 'Energy (eV)'
        • 'e1' (real part)
        • 'e2' (imaginary part)

    Returns:
    - DataFrame containing:
        • summed 'e1' and 'e2'
        • complex dielectric 'e = e1 + 1j·e2'
        • preserved 'Energy (eV)' axis
        • metadata listing each component's source_info
    """
    # 1) Use the energy axis from the first oscillator
    E = df_list[0]["Energy (eV)"].values

    # 2) Sum real and imaginary contributions across all oscillators
    e1_total = sum(df["e1"].values for df in df_list)
    e2_total = sum(df["e2"].values for df in df_list)
    e_total  = e1_total + 1j * e2_total

    # 3) Build the composite DataFrame
    df = pd.DataFrame({
        "Energy (eV)": E,
        "e1":          e1_total,
        "e2":          e2_total,
        "e":           e_total
    })

    # 4) Attach metadata listing each component's model info
    df.attrs["source_info"] = {
        "model":      "Composite",
        "components": [
            d.attrs.get("source_info", {"model": "Unknown"})
            for d in df_list
        ]
    }

    return df